# Compressed Sesing Reconcstruction
## Introduction
This notebook shows basic examples for compressed sensing reconstruction problems with different algorithms.

## Basic reconstruction idea with norm relaxation

Assum the basic $l_0$ minimzation problem from CS:

$$\min_{\mathbf{x}\in\mathbb{R}}\lVert \mathbf{x} \rVert_0\quad\mathrm{s.t.}\quad \lVert \mathbf{y}-\mathbf{A}\mathbf{x}\rVert_2< \epsilon$$

Then, we can relax the $l_0$ "norm" to another norm like the convex $l_1$ norm:

$$\min_{\mathbf{x}\in\mathbb{R}}\lVert \mathbf{x} \rVert_1\quad\mathrm{s.t.}\quad \lVert \mathbf{y}-\mathbf{A}\mathbf{x}\rVert_2 < \epsilon$$

In the noiseless case, the constraint simplifies to $\mathbf{A}\mathbf{x}=\mathbf{y}$. Thus, the solution is at the intersection of the hyperplanes defined by the equation system and the scales unit sphere in the chosen norm. The following code illustrates this with a 2D example. Feasible $\mathbf{x}$ that solve $\mathbf{A}\mathbf{x}=\mathbf{y}$ are on the blue line. This line then touches the scaled unit sphere (green) of different norms at different points (red). Note, that it only leads to a sparse solution for the $l_1$ norm.

In [1]:
%matplotlib widget
import ipywidgets as widgets
from ipywidgets import interact
import matplotlib.pyplot as plt
from matplotlib.patches import Circle, Rectangle
from numpy import linspace
from math import sqrt

def scale_plot_size(factor=1.5):
    import matplotlib as mpl
    default_dpi = mpl.rcParamsDefault['figure.dpi']
    mpl.rcParams['figure.dpi'] = default_dpi*factor

scale_plot_size(0.8)

In [2]:
fig,axs = plt.subplots(1, 4, sharey=True)
fig.suptitle("Illustration of norm effects on the solution")
fig.set_figwidth(12)
fig.set_figheight(3)
#fig.set_tight_layout(True)
fig.canvas.toolbar_visible = False
fig.canvas.header_visible = False
fig.canvas.footer_visible = False
fig.canvas.resizable = False

anno_list=[]
lines_list=[]
sp_space=[]

title_list=('$l_0$ norm', '$l_1$ norm', '$l_2$ norm', '$l_\infty$ norm')
pnt_list=((0,0.5),(0,0.5),(1/5,2/5),(1/3,1/3))
annos=('(0,0.5)','(0,0.5)','(1/5,2/5)','(1/3,1/3)')

x1 = linspace(-1,1,10)

sp_space.append(linspace(0.05,0.5,100))
sp_space.append(linspace(0.05,0.5,100))
sp_space.append(linspace(0.05,1/sqrt(5),100))
sp_space.append(linspace(0.05,1/3,100))

axs[0].set_ylabel('$x_2$')
zero_norm = axs[0].plot([0,0],[0, 0],'g-',[0, 0],[0,0],'g-')

for i in range(4):    
    axs[i].plot(x1,(1-x1)/2)
    axs[i].set_xlim((-1,1))
    axs[i].set_ylim((-1,1))
    axs[i].set_aspect('equal')
    axs[i].set_xlabel('$x_1$')
    axs[i].set_title(title_list[i])
    axs[i].grid(True)
    
    tx,ty=zip(pnt_list[i])
    lines_list.append(axs[i].plot(tx,ty,'rs'))
    lines_list[i][0].set_marker(' ')

    dp = tuple(map(sum, zip(pnt_list[i], (0.05,0.05))))
    anno_list.append(axs[i].annotate(annos[i],dp))
    anno_list[i].set_visible(False)      

def update(change):
    zero_norm[0].set_data([-sp_space[0][change.new], sp_space[0][change.new]],[0, 0])
    zero_norm[1].set_data([0, 0],[-sp_space[0][change.new], sp_space[0][change.new]])
    
    blen=sqrt(sp_space[0][change.new]**2+sp_space[0][change.new]**2)
    axs[1].patches = []
    axs[1].add_patch(Rectangle((0,-sp_space[0][change.new]),blen,blen,45,color='g',alpha=0.1))
    
    axs[2].patches = []
    axs[2].add_patch(Circle((0,0),sp_space[2][change.new],color='g',alpha=0.1))
    
    axs[3].patches = []
    axs[3].add_patch(Rectangle((-sp_space[3][change.new],-sp_space[3][change.new]),2*sp_space[3][change.new],2*sp_space[3][change.new],0,color='g',alpha=0.1))   
    
    if change.new > 95:                      
        for i in range(4):
            lines_list[i][0].set_marker('s')
            anno_list[i].set_visible(True)
    else:
        for i in range(4):
            lines_list[i][0].set_marker(' ')
            anno_list[i].set_visible(False)
    
play = widgets.Play(
    interval=50,
    value=0,
    min=0,
    max=100,
    step=1,
    description="Press play",
    disabled=False
)
play.observe(update, 'value')

slider = widgets.IntSlider()
widgets.jslink((play, 'value'), (slider, 'value'))
widgets.HBox([play, slider])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Reconstruction Example
We now look at asimple example to showcase reconstruction algorithms. Here, we use a random Gaussian matrix of dimension $m\times n$ to subsample and mix the sparse input $\mathbf{x}$. The input is simply 5 non-zero elements set to 1 to make the illustration simple. You can change this easily to random numbers and play around with the parameters. To avoid hard to read plots, we only add a very small amount of noise $\mathbf{n}$ to get the measurements $\mathbf{y}.

In [3]:
import ipywidgets as widgets
from ipywidgets import interact
import matplotlib.pyplot as plt

import numpy.linalg as npl
import numpy.random as npr
import numpy as np

import fastmat as fm
import fastmat.algorithms as fma

# sample number
m = 25 

# sample dimension
n = 100

# sparsity
K = 5

# noise level
snr = 30           # SNR in dB

# define vector matrix shapes for consistency
x_shape = (n,)   # Shape of x matrix
y_shape = (m,)   # Shape of y matrix = shape of y matrix
A_shape = (m,n)   # Shape of A matrix

# Generate the random input the probabilist way...
#x_mean_on = 0     # mean for the on components
#x_var_on = 1      # variance for the on components
#x_on = npr.normal(x_mean_on, np.sqrt(x_var_on), xshape)
#x_on = np.ones(xshape)
#u = np.random.uniform(0, 1, x) < prob_on
#x = x_on*u

# create the ground truth
x = np.zeros(n)
x[npr.choice(range(n), K, replace=0)] = 1 #npr.randn(K)
#x = np.append(np.random.rand(K), np.zeros(n-K))

# Random subsampling matrix
A = npr.normal(0,1/np.sqrt(m),A_shape)

# Measurements without noise
y0 = A.dot(x)

## Generate Gaussian noise
yvar = np.mean(np.abs(y0)**2)
noise_var = yvar*np.power(10, -0.1*snr)
noise = npr.normal(0,np.sqrt(noise_var), y_shape)

# Add noise to measurements
y = y0 + noise

fig, (ax1,ax2,ax3) = plt.subplots(1,3)
fig.canvas.toolbar_visible = False
fig.canvas.header_visible = False
fig.canvas.footer_visible = False
fig.canvas.resizable = False
fig.set_figwidth(12)
fig.tight_layout()
ax1.stem(x,use_line_collection=True)
ax1.set_title('$\mathbf{x}$')
ax2.stem(np.dot(A,x),use_line_collection=True)
ax2.set_title('$\mathbf{A}\mathbf{x}$')
ax3.stem(y,use_line_collection=True)
ax3.set_title('$\mathbf{y}$')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1, '$\\mathbf{y}$')

## Convex Relaxation
First, we take a look at convex relaxation approaches with efficient solvers. The LASSO and Dantzig solvers are from the pacakge [Primal](https://github.com/ShenQianli/primal). The provided algorithms solve both problems for multiple $\lambda$, which is illustrated in the following by an interactive figure. Just change the slider to see how the reconstruction result changes with the parameter.
### BPDN / LASSO

In [6]:
%matplotlib widget
from pyprimal import CompressedSensing

solver = CompressedSensing(A, y)
solver.train()
result = solver.coef()

fig, (ax1,ax2) = plt.subplots(1,2)
fig.canvas.toolbar_visible = False
fig.canvas.header_visible = False
fig.canvas.footer_visible = True
fig.canvas.resizable = False
fig.suptitle('Reconstruction with variable $\lambda$')
fig.set_figwidth(12)
ax1.stem(x,use_line_collection=True)
for a in x.nonzero()[0]:
    ax1.annotate(str(a),(a-1.5,1.012))
ax2.set_title("$\lambda=" + str(result['lambda_list'][0]) + "$")

def draw_result(ind):    
    ax2.clear()
    ax2.set_title("$\lambda=" + str(result['lambda_list'][ind]) + "$")    
    ax2.stem(result['theta_list'][ind],use_line_collection=True)
    
interact(draw_result, ind = widgets.IntSlider(description='$\lambda$-index',
                                              value=0,
                                              min=0,
                                              max=len(result['theta_list'])-1,
                                              step=1))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=0, description='$\\lambda$-index', max=77), Output()), _dom_classes=('wi…

<function __main__.draw_result(ind)>

### Dantzig selector

In [7]:
%matplotlib widget
from pyprimal import Dantzig

solver = Dantzig(A, y)
solver.train()
result = solver.coef()

fig, (ax1,ax2) = plt.subplots(1,2)
fig.canvas.toolbar_visible = False
fig.canvas.header_visible = False
fig.canvas.footer_visible = True
fig.canvas.resizable = False
fig.suptitle('Reconstruction with variable $\lambda$')
fig.set_figwidth(12)
ax1.stem(x,use_line_collection=True)
for a in x.nonzero()[0]:
    ax1.annotate(str(a),(a-1.5,1.012))
ax2.set_title("$\lambda=" + str(result['lambda_list'][0]) + "$")

def draw_result(ind):    
    ax2.clear()
    ax2.set_title("$\lambda=" + str(result['lambda_list'][ind]) + "$")    
    ax2.stem(result['theta_list'][ind],use_line_collection=True)
    
interact(draw_result, ind = widgets.IntSlider(description='$\lambda$-index',
                                              value=0,
                                              min=0,
                                              max=len(result['theta_list'])-1,
                                              step=1))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=0, description='$\\lambda$-index', max=99), Output()), _dom_classes=('wi…

<function __main__.draw_result(ind)>

## Greedy Algorithms
Again, we use the [fastmat](https://github.com/EMS-TU-Ilmenau/fastmat) package for OMP illustration. Now, the number of iterations determines the number of non-zeros entries in the solution directly. By changing the slider, you can vary the number of iterations / non-zeros and take a look at the reconstruction quality.
### OMP

In [8]:
# reconstruct it
A_fm=fm.Matrix(A)
omp = fma.OMP(A_fm, numMaxSteps=1)
x_tilde = omp.process(y)

fig, (ax1,ax2) = plt.subplots(1,2)
fig.canvas.toolbar_visible = False
fig.canvas.header_visible = False
fig.canvas.footer_visible = True
fig.canvas.resizable = False
fig.suptitle('Reconstruction over different $\\alpha$')
fig.set_figwidth(12)
ax1.stem(x,use_line_collection=True)
for a in x.nonzero()[0]:
    ax1.annotate(str(a),(a-1.5,1.012))
ax2.set_title('$K='+str(K)+'$')

def draw_result(ind):    
    ax2.clear()
    ax2.set_title('$K=' + str(ind) + '$')
    omp = fma.OMP(A_fm, numMaxSteps=ind)
    x_tilde = omp.process(y)
    ax2.stem(x_tilde,use_line_collection=True)
        
interact(draw_result, ind = widgets.IntSlider(description='$K$',
                                              value=1,
                                              min=1,
                                              max=n,
                                              step=1))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=1, description='$K$', min=1), Output()), _dom_classes=('widget-interact'…

<function __main__.draw_result(ind)>

## Iterative Thresholding
Next is the class of thresholding algorithms. Here, we use the package [fastmat](https://github.com/EMS-TU-Ilmenau/fastmat). The package also provides advanced Thresholding algorithms, but for the sake of illustration we just show the ISTA algorithm here. Comparable to the illustrations before, we now can play with the threshold of the soft thresholding function by moving the slider to observe the reconstructed vector.
### Soft Thresholding (ISTA)

In [9]:
%matplotlib widget
A_fm=fm.Matrix(A)
ista = fma.ISTA(A_fm, numLambda = 0.01, numMaxSteps = 1000)
x_tilde = ista.process(y)

fig, (ax1,ax2) = plt.subplots(1,2)
fig.canvas.toolbar_visible = False
fig.canvas.header_visible = False
fig.canvas.footer_visible = True
fig.canvas.resizable = False
fig.suptitle('Reconstruction over different $\\alpha$')
fig.set_figwidth(12)
ax1.stem(x,use_line_collection=True)
for a in x.nonzero()[0]:
    ax1.annotate(str(a),(a-1.5,1.012))
ax2.set_title('$\\alpha$')

def draw_result(ind):    
    ax2.clear()
    ax2.set_title('$\\alpha=' + str(ind) + '$')
    ista = fma.ISTA(A_fm, numLambda = ind, numMaxSteps = 1000)
    x_tilde = ista.process(y)
    ax2.stem(x_tilde,use_line_collection=True)
    
interact(draw_result, ind = widgets.FloatSlider(description='$\\alpha$',
                                              value=0.01,
                                              min=0,
                                              max=2,
                                              step=.001))



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=0.01, description='$\\alpha$', max=2.0, step=0.001), Output()), _dom_c…

<function __main__.draw_result(ind)>

## Bayesian Approaches
To show the AMP algorithm, we use the [Vampyre](https://github.com/GAMPTeam/vampyre) package that implements an advanced version of AMP called Generalized Approximate Message Passing (GAMP). For the purpose of this illustration, however, it is identical to AMP.

To fit the probalistic model to the solver, we use a discrete prior with $P_0$ as $k/n$ to model the mean number of non-zero entries approximately.
### Approximate Message Passing

In [10]:
%matplotlib widget
import vampyre as vp

# Define required parameters for the solver that are not used in the non-probablist data model
x_mean_on = 0     # mean for the on components
x_var_on = 1      # variance for the on components
prob_on = K/n      # fraction of components that are *on*

nit = 20  # number of iterations

# define the priors to be mixed (Bernoulli + Gaussian)
est0_off = vp.estim.DiscreteEst(0,1,x_shape)
est0_on = vp.estim.GaussEst(x_mean_on, x_var_on,x_shape)
                   
# Define the mixed prior
est_list = [est0_off, est0_on]
pz0 = np.array([1-prob_on, prob_on])
est0 = vp.estim.MixEst(est_list, w=pz0, name='Input')

# We next define the operator A. In this case the operator is defined by a matrix so we use the MatrixLT class.
Aop = vp.trans.MatrixLT(A,x_shape)

# describe the likelihood function, p(y|z1). Since y=z1+w, we can describe this as a Gaussian estimator.
est1  = vp.estim.GaussEst(y,noise_var,y_shape,name='Output')

# Create the solver
solver = vp.solver.Gamp(est0,est1,Aop,hist_list=['z0', 'zvar0'],nit=nit)
solver.summary()
solver.solve()
x_hat=solver.z0
x_history=solver.hist_dict['z0']

fig, (ax1,ax2) = plt.subplots(1,2)
fig.canvas.toolbar_visible = False
fig.canvas.header_visible = False
fig.canvas.footer_visible = True
fig.canvas.resizable = False
fig.suptitle('Reconstruction over iterations')
fig.set_figwidth(12)
ax1.stem(x,use_line_collection=True)
for a in x.nonzero()[0]:
    ax1.annotate(str(a),(a-1.5,1.012))
ax2.set_title("$Iteration 0")

def draw_result(ind):    
    ax2.clear()
    ax2.set_title("Iteration " + str(ind))
    ax2.stem(x_history[ind],use_line_collection=True)
    
interact(draw_result, ind = widgets.IntSlider(description='Iteration',
                                              value=0,
                                              min=0,
                                              max=len(x_history)-1,
                                              step=1))

est0: Input (Mixture) shape: (100,)
est1: Output (GaussEst) shape: (25,)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=0, description='Iteration', max=19), Output()), _dom_classes=('widget-in…

<function __main__.draw_result(ind)>